# Mock Photon Simulation - Usage

This Notebook serves the purpose to show how you can use the Mock Photon propagation. Let's start with the imports

In [11]:
import sys

sys.path.append('../')

from olympus.event_generation.generators import get_generator
from olympus.configuration.generators import EventGeneratorConfiguration
from olympus.configuration.generators import GenerationConfiguration
from olympus.event_generation.medium import MediumEstimationVariant
from olympus.configuration.generators import UniformSpectrumConfiguration
from ananke.schemas.event import EventType
from olympus.configuration.photon_propagation import MockPhotonPropagatorConfiguration
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.configuration.generators import DatasetConfiguration






### Use Mock Photon Simulation in Pipeline

You should not be obliged to actually call the Simulation itself. Rather than that you should pass it to the generation pipeline. For the optimal pipeline, we need a generator, the hit type we want to generate and a photon propagator, where we will use the mock propagator

In [13]:


from ananke.configurations.presets.detector import single_line_configuration
from olympus.event_generation.generators import generate

photon_propagator_configuration = MockPhotonPropagatorConfiguration(
    resolution=18000,
    medium=MediumEstimationVariant.PONE_OPTIMISTIC
)

configuration = DatasetConfiguration(
    detector=single_line_configuration,
    generators=[
        GenerationConfiguration(
            generator=EventGeneratorConfiguration(
                type=EventType.CASCADE,
                spectrum=UniformSpectrumConfiguration(
                    log_minimal_energy=2.0,
                    log_maximal_energy=5.5
                ),
                source_propagator=photon_propagator_configuration
            ),
            number_of_samples=1
        )
    ],
    data_path="data/nb_mock_photon_usage"
)

collection = generate(configuration)

Let's see what we got!

In [14]:
print('Number of Collection: ', len(collection.records))
records = collection.get_records()
records.df.head()

Number of Collection:  1


,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,energy,length,time,type,particle_id
0,21.08507,-1.847677,432.298426,0.379342,0.795706,-0.472178,-496833911366872595,838.859127,3000.0,0.0,cascade,11


In [15]:
print('Number of Sources: ', len(collection.sources))
sources = collection.get_sources()
sources.df.head()

Number of Sources:  124


,location_x,location_y,location_z,orientation_x,orientation_y,orientation_z,record_id,time,number_of_photons,type
0,21.192823,-1.708394,432.405792,0.379342,0.795706,-0.472178,-496833911366872595,0.336900,832,cherenkov
1,21.205467,-1.515136,432.318085,0.379342,0.795706,-0.472178,-496833911366872595,1.004028,46121,cherenkov
2,21.350779,-1.415070,432.179871,0.379342,0.795706,-0.472178,-496833911366872595,1.671156,394202,cherenkov
3,21.200878,-1.204954,431.989960,0.379342,0.795706,-0.472178,-496833911366872595,2.338284,1548311,cherenkov
4,21.299709,-1.137377,431.759674,0.379342,0.795706,-0.472178,-496833911366872595,3.005413,4015380,cherenkov


In [16]:
print('Number of Hits: ', len(collection.hits))
collection.hits.df.head()

Number of Hits:  2703


,time,string_id,module_id,pmt_id,record_id,type
0,9.385173,0,6,1,-496833911366872595,cascade
1,15.075069,0,6,1,-496833911366872595,cascade
2,10.330508,0,6,1,-496833911366872595,cascade
3,11.374814,0,6,1,-496833911366872595,cascade
4,17.638424,0,6,1,-496833911366872595,cascade


## Use Mock Photon Propagator alone

If you want to call it yourself, you only need sources and the according events.

In [17]:
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from ananke.services.detector import DetectorBuilderService

builder = DetectorBuilderService()
det = builder.get(single_line_configuration)
propagator = MockPhotonPropagator(
    detector=det,
    configuration=photon_propagator_configuration
)
propagator.propagate(collection.get_records(), collection.get_sources())


,time,string_id,module_id,pmt_id,record_id,type
0,9.385173,0,6,1,-496833911366872595,cascade
1,15.075069,0,6,1,-496833911366872595,cascade
2,10.330508,0,6,1,-496833911366872595,cascade
3,11.374814,0,6,1,-496833911366872595,cascade
4,17.638424,0,6,1,-496833911366872595,cascade
